In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search,
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-18 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-18 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-18 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[64, 64], [128, 128], [256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## Using log signature

In [8]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4)]

In [9]:
import signatory

[
    signatory.logsignature_channels(channels, depth)
    for (channels, depth) in log_signature_dimensions_and_sig_depths
]

[406, 385, 406]

### Using UMAP

In [10]:
(
    ffn_logsignature_umap_kfold,
    best_ffn_logsignature_umap_kfold,
    _,
    __,
) = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold_best_model.csv


In [11]:
ffn_logsignature_umap_kfold.groupby(
    ["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1995930/3138393360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (64, 64)   0.1          0.0001         0.495466  0.357446   
                                            0.0005         0.522861  0.352421   
                                            0.0010         0.537739  0.362463   
                               0.5          0.0001         0.567114  0.346219   
                                            0.0005         0.598978  0.334396   
...                                                             ...       ...   
28        2         (512, 512) 0.1          0.0005         0.573499  0.421227   
                                            0.0010         0.559387  0.415112   
                               0.5          0.0001         0.574457  0.421726   
                                            0.0005         0.580013  0.425165   
                                            0.0010         0.568646  0.407322   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (64, 64)   0.1          0.0001          0.359312   
                                            0.0005          0.353064   
                                            0.0010          0.363349   
                               0.5          0.0001          0.352506   
                                            0.0005          0.351271   
...                                                              ...   
28        2         (512, 512) 0.1          0.0005          0.421875   
                                            0.0010          0.413325   
                               0.5          0.0001          0.421256   
                                            0.0005          0.424744   
                                            0.0010          0.409513   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (64, 64)   0.1          0.0001         0.363922   
                                            0.0005         0.354108   
                                            0.0010         0.363620   
                               0.5          0.0001         0.348930   
                                            0.0005         0.345770   
...                                                             ...   
28        2         (512, 512) 0.1          0.0005         0.422968   
                                            0.0010         0.423332   
                               0.5          0.0001         0.422888   
                                            0.0005         0.425732   
                                            0.0010         0.414361   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (64, 64)   0.1          0.0001               0.630700   
                                            0.0005               0.651252   
                                            0.0010               0.653233   
                               0.5          0.0001               0.641244   
                                            0.0005               0.638247   
...                                                                   ...   
28        2         (512, 512) 0.1          0.0005               0.687005   
                                            0.0010               0.678602   
                               0.5          0.0001               0.692571   
                                            0.0005               0.684061   
                                            0.0010               0.674802   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [12]:
best_ffn_logsignature_umap_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.577778,0.432522,"[0.717744316504281, 0.3567362428842505, 0.2230...",0.434162,"[0.7331121833534379, 0.3302881236823612, 0.239...",0.433296,"[0.7030075187969925, 0.38778877887788776, 0.20...",None,0.695568,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.587165,0.430103,"[0.7317073170731707, 0.3303303303303304, 0.228...",0.430284,"[0.7260820045558086, 0.34405719392314565, 0.22...",0.430480,"[0.7374204742625795, 0.31765676567656764, 0.23...",None,0.700385,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.569349,0.420508,"[0.7186629526462396, 0.31574708014498587, 0.22...",0.419123,"[0.7288135593220338, 0.3084185680566483, 0.220...",0.422256,"[0.7087912087912088, 0.3234323432343234, 0.234...",None,0.699743,...,focal,2,True,5,64,790,28,2,umap,True


In [13]:
best_ffn_logsignature_umap_kfold["f1"].mean()

0.42771063470837545

In [14]:
best_ffn_logsignature_umap_kfold["precision"].mean()

0.4278562737505666

In [15]:
best_ffn_logsignature_umap_kfold["recall"].mean()

0.4286774544043956

In [16]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

array([0.72270486, 0.33427122, 0.22615582])

In [17]:
np.stack(best_ffn_logsignature_umap_kfold["precision_scores"]).mean(axis=0)

array([0.72933592, 0.32758796, 0.22664494])

In [18]:
np.stack(best_ffn_logsignature_umap_kfold["recall_scores"]).mean(axis=0)

array([0.7164064 , 0.3429593 , 0.22666667])

### Using random projections

In [19]:
(
    ffn_logsignature_grp_kfold,
    best_ffn_logsignature_grp_kfold,
    _,
    __,
) = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold_best_model.csv


In [20]:
ffn_logsignature_grp_kfold.groupby(
    ["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1995930/2536377135.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (64, 64)   0.1          0.0001         0.597893  0.491853   
                                            0.0005         0.600128  0.498375   
                                            0.0010         0.602043  0.503431   
                               0.5          0.0001         0.608301  0.492738   
                                            0.0005         0.605109  0.498546   
...                                                             ...       ...   
28        2         (512, 512) 0.1          0.0005         0.591188  0.469971   
                                            0.0010         0.568391  0.466581   
                               0.5          0.0001         0.607152  0.475061   
                                            0.0005         0.578416  0.466988   
                                            0.0010         0.563665  0.457007   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (64, 64)   0.1          0.0001          0.486613   
                                            0.0005          0.491002   
                                            0.0010          0.494401   
                               0.5          0.0001          0.491095   
                                            0.0005          0.492358   
...                                                              ...   
28        2         (512, 512) 0.1          0.0005          0.465819   
                                            0.0010          0.459939   
                               0.5          0.0001          0.474691   
                                            0.0005          0.460236   
                                            0.0010          0.449914   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (64, 64)   0.1          0.0001         0.503523   
                                            0.0005         0.513577   
                                            0.0010         0.522275   
                               0.5          0.0001         0.499376   
                                            0.0005         0.510995   
...                                                             ...   
28        2         (512, 512) 0.1          0.0005         0.480186   
                                            0.0010         0.488989   
                               0.5          0.0001         0.476326   
                                            0.0005         0.481219   
                                            0.0010         0.477119   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (64, 64)   0.1          0.0001               0.714783   
                                            0.0005               0.710233   
                                            0.0010               0.704078   
                               0.5          0.0001               0.713177   
                                            0.0005               0.708574   
...                                                                   ...   
28        2         (512, 512) 0.1          0.0005               0.718315   
                                            0.0010               0.710447   
                               0.5          0.0001               0.734746   
                                            0.0005               0.724791   
                                            0.0010               0.718422   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [21]:
best_ffn_logsignature_grp_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.609770,0.475558,"[0.7417550058892814, 0.39578454332552687, 0.28...",0.474672,"[0.7555488902219556, 0.37555555555555553, 0.29...",0.477409,"[0.7284557547715442, 0.4183168316831683, 0.285...",None,0.731214,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.612069,0.476802,"[0.7418648766963374, 0.3940965297167929, 0.294...",0.476738,"[0.7487481590574374, 0.38146718146718145, 0.3]",0.477263,"[0.735106998264893, 0.4075907590759076, 0.2890...",None,0.736191,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.599617,0.472822,"[0.7317577548005908, 0.37374517374517374, 0.31...",0.472663,"[0.7478864734299517, 0.35123367198838895, 0.31...",0.474308,"[0.71631000578369, 0.39933993399339934, 0.3072...",None,0.736834,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True


In [22]:
best_ffn_logsignature_grp_kfold["f1"].mean()

0.4750606366540911

In [23]:
best_ffn_logsignature_grp_kfold["precision"].mean()

0.474690922667774

In [24]:
best_ffn_logsignature_grp_kfold["recall"].mean()

0.47632649615453154

In [25]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

array([0.73845921, 0.38787542, 0.29884728])

In [26]:
np.stack(best_ffn_logsignature_grp_kfold["precision_scores"]).mean(axis=0)

array([0.75072784, 0.3694188 , 0.30392612])

In [27]:
np.stack(best_ffn_logsignature_grp_kfold["recall_scores"]).mean(axis=0)

array([0.72662425, 0.40841584, 0.29393939])